In [ ]:
# %load_ext autoreload
# %autoreload 2

In [1]:
# usual imports
import os
import sys
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.sompz_version.utils import RAIL_SOMPZ_DIR
#from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
import matplotlib.pyplot as plt

In [2]:
# use local repo with local edits
# rail_sompz_repo_develop_list = ["/Users/jmyles/repositories/rail_sompz",
#                                 "/Users/jmyles/repositories/rail_sompz/lib",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10/site-packages"]
# for rail_sompz_repo_develop in rail_sompz_repo_develop_list:
#     sys.path.insert(0, rail_sompz_repo_develop)
from rail.estimation.algos.sompz import SOMPZInformer
from rail.estimation.algos.sompz import SOMPZEstimator

In [3]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [4]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [5]:
from rail.core.utils import find_rail_file
#trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
#deep_data = DS.read_file("training_data", TableHandle, trainFile)
#wide_data = DS.read_file("input_wide_data", TableHandle, testFile)

In [6]:
#datadir = '/global/cfs/projectdirs/desc-pz/users/jmyles/sompz_desc/' 
#hsc_file = os.path.join(datadir, "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-18_common_deep_col_prefix_dropout_placeholder.fits")
#hsc_file = "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-18.fits"
hsc_file = "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-18_common_deep_col_prefix_dropout_placeholder.fits"
deep_data = DS.read_file("training_data", TableHandle, hsc_file)
wide_data = DS.read_file("training_data", TableHandle, hsc_file)

In [7]:
subset = deep_data()[::10]

In [ ]:
#subset[['i_cmodel_flux', 'i_cmodel_fluxerr']]

In [ ]:
#subset['i_cmodel_flux'].value.asarray()

In [8]:
smaller_data = DS.add_data("smaller_data", subset, TableHandle)

In [9]:
bands = ['u','g','r','i','z','y','J','H', 'K']

deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'flux_DEEP_{band}')
    deeperrs.append(f'flux_err_DEEP_{band}')
    zeropts.append(30.)

widebands = []
wideerrs = []  
for band in bands[1:5]:
    widebands.append(f'{band}_cmodel_flux')
    wideerrs.append(f'{band}_cmodel_fluxerr')
    
refband_deep=deepbands[3]
refband_wide=widebands[2]

In [10]:
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, 
                  zero_points_deep=zeropts, 
                  convert_to_flux_deep=False, convert_to_flux_wide=False, 
                  set_threshold_deep=True, thresh_val_deep=1.e-5, 
                  som_shape_wide=(25, 25), som_minerror_wide=0.005,
                  som_take_log_wide=False, som_wrap_wide=False)

In [17]:
som_estimate = SOMPZEstimator.make_stage(name="som_estimator", 
                                         data_path=testFile,
                                         deep_bands=deepbands,
                                         err_deep_bands=deeperrs,  
                                         ref_band_deep=refband_deep,
                                         wide_bands=widebands,
                                         err_wide_bands=wideerrs,    
                                         ref_band_wide=refband_wide,
                                         deep_groupname='', 
                                         hdf5_groupname='',
                                         model="TEST_HSC_model_2023-10-27.pkl", 
                                         **som_params)

initialized model {'deep_som': <rail.estimation.algos.som.NoiseSOM object at 0x154793640>, 'wide_som': <rail.estimation.algos.som.NoiseSOM object at 0x154fad240>, 'deep_columns': ['flux_DEEP_u', 'flux_DEEP_g', 'flux_DEEP_r', 'flux_DEEP_i', 'flux_DEEP_z', 'flux_DEEP_y', 'flux_DEEP_J', 'flux_DEEP_H', 'flux_DEEP_K'], 'deep_err_columns': ['flux_err_DEEP_u', 'flux_err_DEEP_g', 'flux_err_DEEP_r', 'flux_err_DEEP_i', 'flux_err_DEEP_z', 'flux_err_DEEP_y', 'flux_err_DEEP_J', 'flux_err_DEEP_H', 'flux_err_DEEP_K'], 'wide_columns': ['g_cmodel_flux', 'r_cmodel_flux', 'i_cmodel_flux', 'z_cmodel_flux'], 'wide_err_columns': ['g_cmodel_fluxerr', 'r_cmodel_fluxerr', 'i_cmodel_fluxerr', 'z_cmodel_fluxerr']}


In [12]:
nsamp = 100
deep_fluxes_as_ndarray = np.array(subset[deepbands], copy=False).view(('>f4', len(subset[deepbands].dtype.names)))[:nsamp]
deep_flux_errs_as_ndarray = np.array(subset[deeperrs], copy=False).view(('>f4', len(subset[deepbands].dtype.names)))[:nsamp]

wide_fluxes_as_ndarray = np.array(subset[widebands], copy=False).view(('>f4', len(subset[widebands].dtype.names)))[:nsamp]
wide_flux_errs_as_ndarray = np.array(subset[wideerrs], copy=False).view(('>f4', len(subset[widebands].dtype.names)))[:nsamp]

In [15]:
som_estimate.estimate(deep_fluxes_as_ndarray,
                      deep_flux_errs_as_ndarray,
                      wide_fluxes_as_ndarray,
                      wide_flux_errs_as_ndarray,)

hello, run
(100, 9)
(100, 9)
classifying 0
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 8, 9, 9]), array([2, 5, 1, ..., 5, 3, 5]))
False False False True
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 8, 9, 9]), array([2, 5, 1, ..., 5, 3, 5]))
False False False True
classifying 10
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 9, 9, 9]), array([1, 4, 2, ..., 1, 2, 5]))
False False False True
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 9, 9, 9]), array([1, 4, 2, ..., 1, 2, 5]))
False False False True
classifying 20
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 0, ..., 8, 8, 8]), array([1, 2, 4, ..., 1, 2, 3]))
False False False True


/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:340: RuntimeWarning: overflow encountered in divide
  vf = features / errors
/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:360: RuntimeWarning: overflow encountered in exp
  w = np.minimum(np.exp(2 * (vf - 4)), 1000.)
/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:372: RuntimeWarning: invalid value encountered in log
  tmp = w * np.log(2 * vnS) + dn


nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 0, ..., 8, 8, 8]), array([1, 2, 4, ..., 1, 2, 3]))
False False False True
classifying 30
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 0, ..., 9, 9, 9]), array([1, 2, 4, ..., 2, 4, 5]))
False False False True
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 0, ..., 9, 9, 9]), array([1, 2, 4, ..., 2, 4, 5]))
False False False True
classifying 40
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 8, 8, 9]), array([3, 4, 1, ..., 3, 4, 2]))
False False False True
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), array([  0,   0,   0, ..., 511, 511, 511]), array([0, 0, 1, ..., 8, 8, 9]), array([3, 4, 1, ..., 3, 4, 2]))
False False False True
classifying 50
nan tmp at (array([ 0,  0,  0, ..., 40, 40, 40]), 

/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:360: RuntimeWarning: overflow encountered in multiply
  w = np.minimum(np.exp(2 * (vf - 4)), 1000.)


ValueError: ('Wrong shape for initialize ndarray', (625, 4))